# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [2]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [3]:
# headers definition
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Safari/605.1.15" }  


In [4]:
# Your code

json = requests.get(url, headers=headers).json()
normalized_data = json_normalize(json)
normalized_data_articles = json_normalize(normalized_data["articles"][0])
normalized_data_articles.head()

,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount
0,LE223A054-Q12,710 SUPER SKINNY - Jeans Skinny - meteorite,"[3a, 4a, 6a, 8a, 10a, 12a, 14a, 16a]",levisr-710-super-skinny-jeans-skinny-le223a054...,[{'path': 'LE/22/3A/05/4Q/12/LE223A054-Q12@4.j...,Levi's®,False,"[{'sku': 'LE223A054-Q12', 'url_key': 'levisr-7...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"44,95 €","35,95 €",True,False,True,False,NaN
1,QU124L01Y-M11,SCALY YOUTH - Veste mi-saison - dark green,"[8a, 9a, 12a, 14a, 16a]",quiksilver-scaly-youth-veste-dhiver-qu124l01y-m11,[{'path': 'QU/12/4L/01/YM/11/QU124L01Y-M11@6.j...,Quiksilver,False,"[{'sku': 'QU124L01Y-M11', 'url_key': 'quiksilv...","[{'key': 'discountRate', 'value': 'Jusqu’à -50...",clothing,[],"65,99 €","33,00 €",True,False,True,False,NaN
2,QU124L020-Q11,BICHENO BLOOM YOUTH - Veste mi-saison - black,"[8a, 9a, 12a, 14a, 16a]",quiksilver-bicheno-bloom-youth-veste-dhiver-bl...,[{'path': 'QU/12/4L/02/0Q/11/QU124L020-Q11@8.j...,Quiksilver,False,"[{'sku': 'QU124L020-Q11', 'url_key': 'quiksilv...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"129,99 €","103,99 €",False,False,False,False,NaN
3,NI126K00N-K11,HOODIE CLUB - veste en sweat zippée - midnight...,"[6-7a, 8-9a, 10-11a, 12-13a, 14a]",nike-sportswear-hoodie-club-veste-en-sweat-zip...,[{'path': 'NI/12/6K/00/NK/11/NI126K00N-K11@7.j...,Nike Sportswear,False,"[{'sku': 'NI126K00N-K11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -30...",clothing,[],"38,95 €","27,25 €",True,False,True,False,NaN
4,GP024L03G-K11,BABY - Veste d'hiver - bristol blue,"[0-6m, 6-12m, 12-18m]",gap-baby-veste-dhiver-bristol-blue-gp024l03g-k11,[{'path': 'GP/02/4L/03/GK/11/GP024L03G-K11@8.j...,GAP,False,"[{'sku': 'GP024L03G-K11', 'url_key': 'gap-baby...","[{'key': 'discountRate', 'value': '-35%', 'tra...",clothing,[],"39,95 €","25,95 €",False,False,False,False,NaN


#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [5]:
# Get the total number of pages
total_number_pages = json["pagination"]["page_count"]

# Your code
dataset = pd.DataFrame()
for index_page in range(0, total_number_pages):
    print(index_page)
    url= f'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset={84*index_page}&sort=sale'
    json = requests.get(url, headers=headers).json()
    normalized_data = json_normalize(json)
    normalized_data_articles = json_normalize(normalized_data["articles"][0])
    dataset = dataset.append(normalized_data_articles)
    
dataset.head()
    

0
1
2
3


/Users/koide/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259


,amount,brand_name,delivery_promises,family_articles,flags,is_premium,media,name,price.base_price,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,url_key
0,NaN,adidas Originals,[],"[{'sku': 'AD116D007-A11', 'url_key': 'adidas-o...","[{'key': 'discountRate', 'value': 'Jusqu’à -30...",False,[{'path': 'AD/11/6D/00/7A/11/AD116D007-A11@12....,STAN SMITH - Baskets basses - white,NaN,False,True,True,False,"54,95 €","38,45 €",shoe,"[28, 29, 30, 31, 32, 33, 34, 35, 28 1/2, 30 1/...",AD116D007-A11,adidas-originals-stan-smith-baskets-basses-bla...
1,NaN,adidas Originals,[],"[{'sku': 'AD126L00J-Q11', 'url_key': 'adidas-o...","[{'key': 'discountRate', 'value': 'Jusqu’à -35...",False,[{'path': 'AD/12/6L/00/JQ/11/AD126L00J-Q11@8.j...,JACKET - Veste d'hiver - black/white,NaN,False,True,True,False,"64,95 €","41,95 €",clothing,"[3-4a, 4-5a, 5-6a, 6-7a, 7-8a]",AD126L00J-Q11,adidas-originals-jacket-veste-dhiver-blackwhit...
2,NaN,Nike Sportswear,[],"[{'sku': 'NI116D04L-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'NI/11/6D/04/LQ/11/NI116D04L-Q11@8.j...,COURT BOROUGH MID BOOT WINTERIZED - Chaussures...,NaN,False,True,True,False,"54,95 €","43,95 €",shoe,"[28, 28.5, 29.5, 30, 31.5, 32, 33.5, 34, 35]",NI116D04L-Q11,nike-sportswear-court-borough-mid-boot-winteri...
3,NaN,Levi's®,[],"[{'sku': 'LE223K01U-C11', 'url_key': 'levisr-k...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'LE/22/3K/01/UC/11/LE223K01U-C11@6.j...,KEY ITEM LOGO CREW - Sweatshirt - grey heather,NaN,False,False,False,False,"38,95 €","31,15 €",clothing,"[4a, 5a, 8a, 10a, 12a, 14a, 16a]",LE223K01U-C11,levisr-key-item-logo-crew-sweatshirt-le223k01u...
4,NaN,Nike Sportswear,[],"[{'sku': 'NI113D091-A11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'NI/11/3D/09/1A/11/NI113D091-A11@8.j...,AIR MAX 200 - Baskets basses - white/metallic...,NaN,False,True,True,False,"109,95 €","87,95 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5]",NI113D091-A11,nike-sportswear-air-max-200-baskets-basses-whi...


In [6]:
dataset.shape

(21837, 19)

#### Display the trending brand in DataFrame

In [10]:
# your code

trending_brand = dataset.groupby(["brand_name"]).size()
trending_brand.sort_values(ascending=False).to_frame()

,0
brand_name,
Name it,1013
GAP,701
Esprit,676
Boboli,667
Benetton,528
...,...
Kipling,1
HOMEBASE,1
Eastpak,1


#### Display the brand with maximal total discount (sum of discounts on all goods)

In [ ]:
#Our data is still text. Convert prices into numbers.
# your code

# Max total discount = original - promotional



#### Display the brands without discount at all

In [ ]:
# your code

trending_brand = dataset.groupby(["brand_name", ""]).size()
trending_brand.sort_values(ascending=False).to_frame()